In [1]:
import org.apache.spark.SparkConf
import org.apache.spark.SparkContext
import org.apache.spark.rdd.RDD

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.0.3:4040
SparkContext available as 'sc' (version = 3.0.1, master = local[*], app id = local-1605014841241)
SparkSession available as 'spark'


import org.apache.spark.SparkConf
import org.apache.spark.SparkContext
import org.apache.spark.rdd.RDD


## Class definition

In [2]:
class Shingle {
    
    def createshingles(path: String, k:Int) : RDD[(String, Set[Int])] = {
        val files = sc.wholeTextFiles(path,1).map({ case (path, text) => (path.split("/").last, text) })
                                            .mapValues(_.toLowerCase)
                                            .mapValues(_.replace(",", ""))
                                            .mapValues(_.replace(".", ""))
                                            .mapValues(_.replace("?", ""))
                                            .mapValues(_.replace("!", ""))
                                            .mapValues(_.replace(")", ""))
                                            .mapValues(_.replace("(", ""))    
        val shingles = files.flatMapValues { case (str) => str.sliding(k,1)
        }  
        print(shingles.collect)
        return shingles.mapValues(str => str.hashCode()).groupByKey().mapValues(x => x.toSet)
    }
}


class CompareSet {
  def JaccardSimilarity(set1: Set[(Int)], set2: Set[(Int)]) : Double = {
    return ((set1.intersect(set2).size).toDouble / (set1.union(set2).size)).toDouble
  }
}

implicit def bool2int(b:Boolean) = if (b) 1 else 0
import scala.util.control._

class MinHashing {
    
  def create_chrMatrix(shingles_set: Array[(Int)], set: Array[(Int)]) : Array[(Int)] = {
    val chr_col = Array.fill(shingles_set.size)(0)
      for (i <- 0 to shingles_set.size -1){
        chr_col(i) = (set contains shingles_set(i))*1
      }
    return chr_col
  }
    
  def h(x: Int, a:Int, rows:Int) : Int =  {
        return (((x+1)*a)+1)%rows
    }

  def create_signature(shingles_set: Array[(Int)], set: Array[(Int)], permutations: Int) : Array[(Int)] = {
      val chr_col = create_chrMatrix(shingles_set, set)
      val signature = Array.fill(permutations)(0)
      val rows = chr_col.size
      val inloop = new Breaks
          for (a <- 1 to permutations){
              inloop.breakable{
              for (x <- 0 to chr_col.size -1){
                  if (chr_col(h(x,a,rows))==1){
                      signature(a-1) = x+1
                      inloop.break
                      }
                  }
              }
          }
      return signature
  }
    
}

class CompareSignatures {
    
   def similarity(signature0: Array[(Int)], signature1: Array[(Int)]) : Double = {
       val sign0 = signature0.toList
       val sign1 = signature1.toList
       return (sign0.zip(sign1).filter(x => x._1 == x._2).size).toDouble/signature0.size.toDouble
    } 
}

import scala.collection.mutable.ListBuffer

class LocSensHashing{
    
    def candidate_pairs(signatures: Vector[(String, Array[Int])], r: Int) : Vector[(String, String)] = {
        
        val loc_hash = for (i <- 0 to signatures.size -1) yield((signatures(i)._1,
                                                (for (x <- (signatures(i)._2.toList).grouped(r)) yield(x.hashCode())).toList))
        
        val candidates = new ListBuffer[(String, String)]()
        
        for (i <- 0 to signatures.size -1){
            for (j <- i+1 to signatures.size -1){
               if ((loc_hash(i)._2.zip(loc_hash(j)._2).filter(x => x._1 == x._2).size) > 0){
                   candidates += ((signatures(i)._1, signatures(j)._1))
                   //println(signatures(i)._1, signatures(j)._1)
               }
            }
        }
        return candidates.toVector
    }
}

defined class Shingle
defined class CompareSet
bool2int: (b: Boolean)Int
import scala.util.control._
defined class MinHashing
defined class CompareSignatures
import scala.collection.mutable.ListBuffer
defined class LocSensHashing


---

In [3]:
// parameters

val path = "../data/*_taska.txt"
val k = 7

path: String = ../data/*_taska.txt
k: Int = 7


In [4]:
// Create shingles

val shing = new Shingle
val shingles = shing.createshingles(path,k)
val s = shingles.collect()

[Lscala.Tuple2;@64551e0

shing: Shingle = Shingle@537fa12f
shingles: org.apache.spark.rdd.RDD[(String, Set[Int])] = MapPartitionsRDD[13] at mapValues at <console>:29
s: Array[(String, Set[Int])] = Array((g0pE_taska.txt,Set(1024328849, -809942811, 300626, 450333591, -1110351275, 2057428624, 1237394832, -719776277, 877982873, -1509224606, -1348893292, -422012319, 1772178932, -974723696, -604869464, 1718732402, -1897320720, 2129320190, 1217550084, -1351981020, -1542092980, -1956689524, 1979091318, -679719782, -887312734, 667020100, 254387060, 735439297, 1219629894, 1930547162, 1195890950, -1922170827, 833328706, -1911224770, 700148195, -493883412, -1433668758, -1494609322, -1191026030, 1361461549, -1311872163, 1606945631, 939821247, 1216975005, 254386680, 1020649433, 280184718, -77600629, -421994171, 1998498434, -...


In [5]:
// Compute similarity

val similarity = new CompareSet
val set_size = s.size-1

for (i <- 0 to set_size){
    for (j <- i+1 to set_size){
        val sim = similarity.JaccardSimilarity(s(i)._2,s(j)._2)
        println("("+s(i)._1+","+s(j)._1+"):"+sim)
    }
}

(g0pE_taska.txt,g0pC_taska.txt):0.15293080652269722
(g0pE_taska.txt,g1pA_taska.txt):0.05274000824062629
(g0pE_taska.txt,g3pC_taska.txt):0.378978978978979
(g0pE_taska.txt,g0pD_taska.txt):0.5091230135373749
(g0pE_taska.txt,g0pB_taska.txt):0.04813432835820895
(g0pE_taska.txt,g4pD_taska.txt):0.22704199353945548
(g0pE_taska.txt,g3pA_taska.txt):0.06424344885883347
(g0pE_taska.txt,g1pB_taska.txt):0.041072200605274535
(g0pE_taska.txt,g4pC_taska.txt):0.8371958285052143
(g0pE_taska.txt,g2pE_taska.txt):0.5117101193106496
(g0pE_taska.txt,g2pA_taska.txt):0.0370631839039887
(g0pE_taska.txt,orig_taska.txt):0.8938515081206496
(g0pE_taska.txt,g2pB_taska.txt):0.03643410852713178
(g0pE_taska.txt,g4pE_taska.txt):0.09850107066381156
(g0pE_taska.txt,g0pA_taska.txt):0.05443548387096774
(g0pE_taska.txt,g3pB_taska.txt):0.05779889152810768
(g0pE_taska.txt,g2pC_taska.txt):0.36591606960081885
(g0pE_taska.txt,g4pB_taska.txt):0.0325678496868476
(g0pE_taska.txt,g1pD_taska.txt):0.23805309734513275
(g0pC_taska.txt,g1p

(g2pA_taska.txt,g3pB_taska.txt):0.062146892655367235
(g2pA_taska.txt,g2pC_taska.txt):0.03784013605442177
(g2pA_taska.txt,g4pB_taska.txt):0.041280148423005564
(g2pA_taska.txt,g1pD_taska.txt):0.042173560421735604
(orig_taska.txt,g2pB_taska.txt):0.03663551401869159
(orig_taska.txt,g4pE_taska.txt):0.09449465899753492
(orig_taska.txt,g0pA_taska.txt):0.05357142857142857
(orig_taska.txt,g3pB_taska.txt):0.05803741886216113
(orig_taska.txt,g2pC_taska.txt):0.3608652900688299
(orig_taska.txt,g4pB_taska.txt):0.03459372485921158
(orig_taska.txt,g1pD_taska.txt):0.22754237288135593
(g2pB_taska.txt,g4pE_taska.txt):0.04275653923541248
(g2pB_taska.txt,g0pA_taska.txt):0.08426966292134831
(g2pB_taska.txt,g3pB_taska.txt):0.06549364613880743
(g2pB_taska.txt,g2pC_taska.txt):0.0436241610738255
(g2pB_taska.txt,g4pB_taska.txt):0.0825136612021858
(g2pB_taska.txt,g1pD_taska.txt):0.038271049076992344
(g4pE_taska.txt,g0pA_taska.txt):0.04895833333333333
(g4pE_taska.txt,g3pB_taska.txt):0.050202839756592295
(g4pE_task

similarity: CompareSet = CompareSet@2fa19263
set_size: Int = 19


In [6]:
//MinHashing

// create common set of shingles
val shingles_list = scala.util.Random.shuffle(
                    ((for (x <- 0 to set_size) yield (s(x)._2).toList).flatten).distinct).toArray

// create the signature matrix
val minhash = new MinHashing
val permutations = 500

val sign_list = (for (x <- 0 to set_size) 
                 yield ((s(x)._1,minhash.create_signature(shingles_list, (s(x)._2).toArray, permutations))))

shingles_list: Array[Int] = Array(-676003284, -741324038, 1835436054, 1958058841, 1495855149, 880248580, 1982507541, -1338976461, 1737030804, 1884280085, 1215648375, -62157066, -1609053654, 2052219601, 735204148, 1062454933, 1566069978, 1082361173, 735740774, 1030175919, 279998042, -1413252334, -1493339062, 1034902516, -1354641915, 592571771, 1979079784, 1527550525, 748863544, 1083684932, -1315402357, -1955377442, 1217770368, -1751076621, -50476630, 2092693143, -1494778981, 1217600726, -754476856, -1614129117, -1922170682, 1253817676, -1493338854, -1212807901, -132939089, 2119677723, 562965038, 1608433632, -746065241, 735774845, -1177326501, -1063713245, 133823211, -1351985807, 1590387683, -1668934408, 5850148, 1769568470, -1884253188, 1981488898, 891007317, 1979100060, 343586639, 45731...


In [7]:
// Signatures comparison

val compSign = new CompareSignatures

for (i <- 0 to set_size){
    for (j <- i+1 to set_size){
        val sim = compSign.similarity(sign_list(i)._2,sign_list(j)._2)
        println("("+sign_list(i)._1+","+sign_list(j)._1+"):"+sim)
    }
}

(g0pE_taska.txt,g0pC_taska.txt):0.188
(g0pE_taska.txt,g1pA_taska.txt):0.04
(g0pE_taska.txt,g3pC_taska.txt):0.37
(g0pE_taska.txt,g0pD_taska.txt):0.586
(g0pE_taska.txt,g0pB_taska.txt):0.056
(g0pE_taska.txt,g4pD_taska.txt):0.234
(g0pE_taska.txt,g3pA_taska.txt):0.046
(g0pE_taska.txt,g1pB_taska.txt):0.034
(g0pE_taska.txt,g4pC_taska.txt):0.85
(g0pE_taska.txt,g2pE_taska.txt):0.556
(g0pE_taska.txt,g2pA_taska.txt):0.052
(g0pE_taska.txt,orig_taska.txt):0.904
(g0pE_taska.txt,g2pB_taska.txt):0.048
(g0pE_taska.txt,g4pE_taska.txt):0.074
(g0pE_taska.txt,g0pA_taska.txt):0.072
(g0pE_taska.txt,g3pB_taska.txt):0.068
(g0pE_taska.txt,g2pC_taska.txt):0.434
(g0pE_taska.txt,g4pB_taska.txt):0.032
(g0pE_taska.txt,g1pD_taska.txt):0.23
(g0pC_taska.txt,g1pA_taska.txt):0.07
(g0pC_taska.txt,g3pC_taska.txt):0.096
(g0pC_taska.txt,g0pD_taska.txt):0.168
(g0pC_taska.txt,g0pB_taska.txt):0.094
(g0pC_taska.txt,g4pD_taska.txt):0.12
(g0pC_taska.txt,g3pA_taska.txt):0.08
(g0pC_taska.txt,g1pB_taska.txt):0.072
(g0pC_taska.txt,g4p

compSign: CompareSignatures = CompareSignatures@45fa2c59


In [8]:
// Local-Sensitive Hashing

val t = 0.8
val b = 50
val r = 10

val LSH = new LocSensHashing
val cand = LSH.candidate_pairs(sign_list.toVector, r);

t: Double = 0.8
b: Int = 50
r: Int = 10
LSH: LocSensHashing = LocSensHashing@1d3f3d94
cand: Vector[(String, String)] = Vector((g0pE_taska.txt,g4pC_taska.txt), (g0pE_taska.txt,orig_taska.txt), (g4pC_taska.txt,orig_taska.txt))
